# Imports
When you reach the line:

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit

Press the stop button on the cell and it will continue running

In [ ]:
## Install Packages for Rendering
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install imageio==2.4.1

  Using cached imageio-2.4.1-py3-none-any.whl
  Attempting uninstall: imageio
    Found existing installation: imageio 2.34.0
    Uninstalling imageio-2.34.0:
      Successfully uninstalled imageio-2.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moviepy 1.0.3 requires imageio<3.0,>=2.5; python_version >= "3.4", but you have imageio 2.4.1 which is incompatible.


In [ ]:
!pip install git+https://github.com/MLivanos/colabgymrender.git

  Cloning https://github.com/MLivanos/colabgymrender.git to /tmp/pip-req-build-6hbvtpjp
  Running command git clone --filter=blob:none --quiet https://github.com/MLivanos/colabgymrender.git /tmp/pip-req-build-6hbvtpjp
  Resolved https://github.com/MLivanos/colabgymrender.git to commit 8526ad9deec50a445e63b8faf2b894d3b0cd7cd2
  Preparing metadata (setup.py) ... done
  Using cached imageio-2.34.0-py3-none-any.whl (313 kB)
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [ ]:
## Install ROMs
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

--2024-03-16 00:33:40--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 212.83.163.6
Connecting to www.atarimania.com (www.atarimania.com)|212.83.163.6|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.atarimania.com/roms/Roms.rar [following]
--2024-03-16 00:33:40--  https://www.atarimania.com/roms/Roms.rar
Connecting to www.atarimania.com (www.atarimania.com)|212.83.163.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19612325 (19M) [application/octet-stream]
Saving to: ‘Roms.rar.5’

Roms.rar.5          100%[===================>]  18.70M  48.7MB/s    in 0.4s    

2024-03-16 00:33:41 (48.7 MB/s) - ‘Roms.rar.5’ saved [19612325/19612325]

mkdir: cannot create directory ‘/content/ROM/’: File exists

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/Roms.rar


Would you like to replace the existing file /content/ROM/1

In [ ]:
#!wget https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1
#!pip install gym[atari,accept-rom-license]==0.21.0
!pip install ale-py
!pip install gym[atari,accept-rom-license]
# Example for generating the game replay
import gym
from colabgymrender.recorder import Recorder

#env = gym.make("PongNoFrameskip-v4")
env = gym.make("MsPacmanNoFrameskip-v4")
directory = './video'
env = Recorder(env, directory)

observation = env.reset()
terminal = False
while not terminal:
  action = env.action_space.sample()
  observation, reward, terminal, info = env.step(action)
env.play()

/bin/bash: line 1: pip install ale-py: command not found


  np.bool8: (False, True),

  from scipy.ndimage.filters import sobel

  deprecation(

  deprecation(

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(

  logger.warn(

  logger.deprecation(

  if not isinstance(done, (bool, np.bool8)):



Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



t:  98%|█████████▊| 1914/1956 [00:01<00:00, 1613.22it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file ./video/1710549240.5970542.mp4, 100800 bytes wanted but 0 bytes read,at frame 1955/1956, at time 65.17/65.17 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

                                                                  

Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


# Layers.py

In [ ]:
# layers.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class NoisyLinear(nn.Module):
    def __init__(self, in_features, out_features, use_cuda, std_init=0.4):
        super(NoisyLinear, self).__init__()

        self.use_cuda     = use_cuda
        self.in_features  = in_features
        self.out_features = out_features
        self.std_init     = std_init

        self.weight_mu    = nn.Parameter(torch.FloatTensor(out_features, in_features))
        self.weight_sigma = nn.Parameter(torch.FloatTensor(out_features, in_features))
        self.register_buffer('weight_epsilon', torch.FloatTensor(out_features, in_features))

        self.bias_mu    = nn.Parameter(torch.FloatTensor(out_features))
        self.bias_sigma = nn.Parameter(torch.FloatTensor(out_features))
        self.register_buffer('bias_epsilon', torch.FloatTensor(out_features))

        self.reset_parameters()
        self.reset_noise()

    def forward(self, x):
        if self.use_cuda:
            weight_epsilon = self.weight_epsilon.cuda()
            bias_epsilon   = self.bias_epsilon.cuda()
        else:
            weight_epsilon = self.weight_epsilon
            bias_epsilon   = self.bias_epsilon

        if self.training:
            weight = self.weight_mu + self.weight_sigma.mul(Variable(weight_epsilon))
            bias   = self.bias_mu   + self.bias_sigma.mul(Variable(bias_epsilon))
        else:
            weight = self.weight_mu
            bias   = self.bias_mu

        return F.linear(x, weight, bias)

    def reset_parameters(self):
        mu_range = 1 / math.sqrt(self.weight_mu.size(1))

        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(self.std_init / math.sqrt(self.weight_sigma.size(1)))

        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(self.std_init / math.sqrt(self.bias_sigma.size(0)))

    def reset_noise(self):
        epsilon_in  = self._scale_noise(self.in_features)
        epsilon_out = self._scale_noise(self.out_features)

        self.weight_epsilon.copy_(epsilon_out.ger(epsilon_in))
        self.bias_epsilon.copy_(self._scale_noise(self.out_features))

    def _scale_noise(self, size):
        x = torch.randn(size)
        x = x.sign().mul(x.abs().sqrt())
        return x

# Wrappers.py

In [ ]:
import numpy as np
from collections import deque
import gym
from gym import spaces
import cv2
cv2.ocl.setUseOpenCL(False)

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.integers(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def reset(self):
        return self.env.reset()

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255,
            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]

class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0], shp[1], shp[2] * k), dtype=np.uint8)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=2)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

def make_atari(env_id):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env



class ImageToPyTorch(gym.ObservationWrapper):
    """
    Image shape to num_channels x weight x height
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]), dtype=np.uint8)

    def observation(self, observation):
        return np.swapaxes(observation, 2, 0)


def wrap_pytorch(env):
    return ImageToPyTorch(env)

# DQN.py

In [ ]:
from collections import deque
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.autograd as autograd
import math, random
USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

class QLearner(nn.Module):
    def __init__(self, env, num_frames, batch_size, gamma, replay_buffer):
        super(QLearner, self).__init__()
        self.batch_size = batch_size
        self.gamma = gamma
        self.num_frames = num_frames
        self.replay_buffer = replay_buffer
        self.env = env
        self.input_shape = self.env.observation_space.shape
        self.num_actions = self.env.action_space.n
        self.features = nn.Sequential(
            nn.Conv2d(self.input_shape[0], 32, kernel_size=8, stride=4), #hidden layer 1, 20x20
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), #hidden layer 2, 9x9
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), #hidden layer 3, 7x7
            nn.ReLU()
        )
        self.fc = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, self.num_actions)
        )

    def forward(self, x): #get q values of a state
        x = self.features(x) #pass the current input through a series of layers to extract features
        x = x.view(x.size(0), -1) #reshape (flatten) output tensor
        x = self.fc(x) #pass the flattened tensor through a series of linear perceptrons
        return x

    def feature_size(self):
            return self.features(autograd.Variable(torch.zeros(1, *self.input_shape))).view(1, -1).size(1)

    def act(self, state, epsilon):
        if random.random() > epsilon:
            state = Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), requires_grad=True)
            # TODO: Given state, you should write code to get the Q value and chosen action

            q_values = self.forward(state) #get q values of state

            # Convert Q-values tensor to a numpy array
            #q_values = q_values.data.cpu().numpy()

            # Choose action with the highest Q-value
            #action = np.argmax(q_values)
            q_value = q_values.max(1)[1]

            return q_value.item() #return single value within the tensor


        else:
            action = random.randrange(self.env.action_space.n)
        return action

    def copy_from(self, target):
        self.load_state_dict(target.state_dict())


def compute_td_loss(model, target_model, batch_size, gamma, replay_buffer):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)
    state = Variable(torch.FloatTensor(np.float32(state)))
    next_state = Variable(torch.FloatTensor(np.float32(next_state)), requires_grad=True)
    action = Variable(torch.LongTensor(action))
    reward = Variable(torch.FloatTensor(reward))
    done = Variable(torch.FloatTensor(done))
    # TODO: IMPLEMENT LOSS

    #get all (predicted) q values for every single sampled state (for all actions)
    predicted_q_values = model(state)

    #get THE (predicted) Q value for every single state, action pair
    predicted_q_value = predicted_q_values[range(batch_size), action] #get the q value of the performed action at every sampled frame
    #predicted_q_value = predicted_q_values.gather(1, action.unsqueeze(1)).squeeze(1)

    #get all (expected) q values for all next states of the sampled current states
    next_q_values = target_model(next_state)

    #get the target q values of the sampled frames
    #[0] gets the max values. [1] gets the argmax
    #if done is true, we are at the output unit (no more next state)
    target_q_value = reward + gamma * next_q_values.max(1)[0] * (1 - done)

    #calculate loss
    loss = nn.MSELoss(reduction="sum")(target_q_value,  predicted_q_value)

    return loss

    '''
    # Get Q-values for the current state from the model
    q_values = model(state)

    # Get Q-values for the next state from the target model
    next_q_values = target_model(next_state)

    # Gather the Q-values corresponding to the actions taken
    q_value = q_values.gather(1, action.unsqueeze(1)).squeeze(1)

    # Compute the target Q-value using the Bellman equation
    target_q_value = reward + gamma * next_q_values.max(1)[0] * (1 - done)

    # Calculate the TD error (difference between predicted and target Q-values)
    td_error = nn.MSELoss()(q_value, target_q_value.detach())

    return td_error'''


class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        state = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        # TODO: Randomly sampling data with specific batch size from the buffer (batch size = # of frames we learn from at once)

        # Randomly sample a batch of experiences from the replay buffer
        batch = random.sample(self.buffer, batch_size)

        # Unpack the batch into separate variables
        state, action, reward, next_state, done = zip(*batch)

        # combine all arrays within states/ next_states into a single array
        state = np.concatenate(state)
        next_state = np.concatenate(next_state)

        return state, action, reward, next_state, done

    def __len__(self):
        return len(self.buffer)

# Test Your Model

Note: Test is intentionally before train because you are provided a pretrained - but imperect - model. You can go back and use this code to test your model after you train it.

In [ ]:
# Test_dqn
import sys
from colabgymrender.recorder import Recorder
import math, random
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
USE_CUDA = torch.cuda.is_available()

env_id = "PongNoFrameskip-v4"
env = make_atari(env_id)
env = wrap_deepmind(env)
env = wrap_pytorch(env)
directory = './video'
env = Recorder(env, directory)

num_frames = 1000000
batch_size = 32
gamma = 0.99

replay_initial = 10000
replay_buffer = ReplayBuffer(100000)
# specify the path of the testing model
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
pthname = '/content/drive/My Drive/Colab_Notebooks/Assignments/Deep_Q_Network/PongNoFrameskip-v4-model.pth'

model = QLearner(env, num_frames, batch_size, gamma, replay_buffer)
model.load_state_dict(torch.load(pthname, map_location='cpu'))
model.eval()
if USE_CUDA:
    model = model.cuda()
    print("Using cuda")

env.seed(1)
state = env.reset()
done = False
games_won = 0

while not done:
    action = model.act(state, 0)
    state, reward, done, _ = env.step(action)
    if reward != 0:
        print(reward)
    if reward == 1:
        games_won += 1

print("Points Won: {}".format(games_won))
env.play()
try:
    sys.exit(0)
except:
    pass

Mounted at /content/drive
Using cuda
-1.0
-1.0
-1.0
-1.0
1.0
1.0
1.0
-1.0
1.0
1.0
1.0
-1.0
1.0
1.0
1.0
-1.0
1.0
1.0
1.0
-1.0
-1.0
-1.0
-1.0
1.0
1.0
-1.0
1.0
1.0
-1.0
-1.0
1.0
-1.0
1.0
1.0
1.0
1.0
Points Won: 21
Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


# Train your Model

In [ ]:
# run_dqn (train your model)
import math, random
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
USE_CUDA = torch.cuda.is_available()

env_id = "PongNoFrameskip-v4"
env = make_atari(env_id)
env = wrap_deepmind(env)
env = wrap_pytorch(env)

num_frames = 1000000
batch_size = 32
gamma = 0.99
record_idx = 10000

replay_initial = 10000
replay_buffer = ReplayBuffer(100000)
model = QLearner(env, num_frames, batch_size, gamma, replay_buffer)

# specify the path of the testing model
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#pthname = '/content/drive/My Drive/Colab_Notebooks/Assignments/Deep_Q_Network/model_pretrained.pth'
pthname = '/content/drive/My Drive/Colab_Notebooks/Assignments/Deep_Q_Network/saves_2/620000_model.pth' #continue off from model with 620k frames of training
model.load_state_dict(torch.load(pthname, map_location='cpu'))

target_model = QLearner(env, num_frames, batch_size, gamma, replay_buffer)
target_model.copy_from(model)

optimizer = optim.Adam(model.parameters(), lr=0.00001)
if USE_CUDA:
    model = model.cuda()
    target_model = target_model.cuda()
    print("Using cuda")

epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 30000
epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

losses = []
all_rewards = []
episode_reward = 0

state = env.reset()

for frame_idx in range(620000, num_frames + 1): #modified: continue from frame 620k

    # TODO: PERIODICALLY SAVE MODEL
    if frame_idx % 20000 == 0: #save every 20kth frame
        torch.save(model.state_dict(), f"/content/drive/My Drive/Colab_Notebooks/Assignments/Deep_Q_Network/saves_2/{frame_idx}_model.pth")


    epsilon = epsilon_by_frame(frame_idx)
    action = model.act(state, epsilon)

    next_state, reward, done, _ = env.step(action)
    replay_buffer.push(state, action, reward, next_state, done)

    state = next_state
    episode_reward += reward

    if done:
        state = env.reset()
        all_rewards.append((frame_idx, episode_reward))
        episode_reward = 0

    if len(replay_buffer) > replay_initial:
        loss = compute_td_loss(model, target_model, batch_size, gamma, replay_buffer)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append((frame_idx, loss.data.cpu().numpy()))

    if frame_idx % 10000 == 0 and len(replay_buffer) <= replay_initial:
        print('#Frame: %d, preparing replay buffer' % frame_idx)

    if frame_idx % 10000 == 0 and len(replay_buffer) > replay_initial:
        print('#Frame: %d, Loss: %f' % ((frame_idx), np.mean(losses, 0)[1]))
        print('Last-10 average reward: %f' % np.mean(all_rewards[-10:], 0)[1])

    if frame_idx % 50000 == 0:
        target_model.copy_from(model)

# set your model path
saved_path = "/content/drive/My Drive/" + env_id + "-model.pth"
# save your final model
torch.save(model.state_dict(), saved_path)

Mounted at /content/drive


  logger.deprecation(

  if not isinstance(done, (bool, np.bool8)):



Using cuda
#Frame: 620000, preparing replay buffer
#Frame: 630000, Loss: 0.621185
Last-10 average reward: 6.000000
#Frame: 640000, Loss: 0.059358
Last-10 average reward: 2.857143
#Frame: 650000, Loss: 0.048262
Last-10 average reward: 2.400000
#Frame: 660000, Loss: 0.057776
Last-10 average reward: 1.300000
#Frame: 670000, Loss: 0.057691
Last-10 average reward: -0.700000
#Frame: 680000, Loss: 0.057098
Last-10 average reward: -2.300000
#Frame: 690000, Loss: 0.056372
Last-10 average reward: -2.900000
#Frame: 700000, Loss: 0.055618
Last-10 average reward: -2.900000
#Frame: 710000, Loss: 0.061574
Last-10 average reward: -1.600000
#Frame: 720000, Loss: 0.063592
Last-10 average reward: -3.000000
#Frame: 730000, Loss: 0.064141
Last-10 average reward: 0.100000
#Frame: 740000, Loss: 0.064095
Last-10 average reward: -0.600000
#Frame: 750000, Loss: 0.063920
Last-10 average reward: 1.200000
#Frame: 760000, Loss: 0.067287
Last-10 average reward: 0.500000
#Frame: 770000, Loss: 0.067993
Last-10 average

In [ ]:
#save all_reward to new .csv
import csv

end = 180000 #insert here, the last 20kth frame before training stopped
all_rewards_to_save = all_rewards[:end]
path = '/content/drive/My Drive/Colab_Notebooks/Assignments/Deep_Q_Network/saves_2/rewards.csv'
with open(path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(all_rewards_to_save)

In [ ]:
#append all_reward to existing .csv
import csv

start = 620000 #insert here, the frame we started from
end = 1000000 #insert here, the last 20kth frame before training stopped
all_rewards_to_save_ = all_rewards[:(end-start)]
path = '/content/drive/My Drive/Colab_Notebooks/Assignments/Deep_Q_Network/saves_2/rewards.csv'
with open(path, 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(all_rewards_to_save_)